In [1]:
import requests
from typing import Dict, Any, Optional

BASE_URL = "https://berghain.challenges.listenlabs.ai/"  # ← replace with the real host
PLAYER_ID = "5d7389b8-683e-453b-a35f-6d21b1b5e94e"
SCENARIO = 3  # choose 1, 2, or 3

def create_new_game(scenario: int, player_id: str) -> Dict[str, Any]:
    url = f"{BASE_URL}/new-game"
    res = requests.get(url, params={"scenario": scenario, "playerId": player_id}, timeout=30)
    res.raise_for_status()
    data = res.json()
    print("New game:", data["gameId"])
    return data

In [2]:
new_game = create_new_game(SCENARIO, PLAYER_ID)
game_id = new_game["gameId"]
constraints = new_game.get("constraints", [])
stats = new_game.get("attributeStatistics", {})
print(constraints)
print(stats)

New game: a795f96e-6403-439e-9e2c-b9b3c1270f8a
[{'attribute': 'underground_veteran', 'minCount': 500}, {'attribute': 'international', 'minCount': 650}, {'attribute': 'fashion_forward', 'minCount': 550}, {'attribute': 'queer_friendly', 'minCount': 250}, {'attribute': 'vinyl_collector', 'minCount': 200}, {'attribute': 'german_speaker', 'minCount': 800}]
{'relativeFrequencies': {'underground_veteran': 0.6794999999999999, 'international': 0.5735, 'fashion_forward': 0.6910000000000002, 'queer_friendly': 0.04614, 'vinyl_collector': 0.044539999999999996, 'german_speaker': 0.4565000000000001}, 'correlations': {'underground_veteran': {'underground_veteran': 1, 'international': -0.08110175777152992, 'fashion_forward': -0.1696563475505309, 'queer_friendly': 0.03719928376753885, 'vinyl_collector': 0.07223521156389842, 'german_speaker': 0.11188766703422799}, 'international': {'underground_veteran': -0.08110175777152992, 'international': 1, 'fashion_forward': 0.375711059360155, 'queer_friendly': 0.0

In [3]:
from helper import convert_data_to_arrays
stats = convert_data_to_arrays(stats)
marginals, target_corr = stats
print(marginals.tolist())
print(target_corr.tolist())

[0.6794999999999999, 0.5735, 0.6910000000000002, 0.04614, 0.044539999999999996, 0.4565000000000001]
[[1.0, -0.08110175777152992, -0.1696563475505309, 0.03719928376753885, 0.07223521156389842, 0.11188766703422799], [-0.08110175777152992, 1.0, 0.375711059360155, 0.0036693314388711686, -0.03083247098181075, -0.7172529382519395], [-0.1696563475505309, 0.375711059360155, 1.0, -0.0034530926793377476, -0.11024719606358546, -0.3521024461597403], [0.03719928376753885, 0.0036693314388711686, -0.0034530926793377476, 1.0, 0.47990640803167306, 0.04797381132680503], [0.07223521156389842, -0.03083247098181075, -0.11024719606358546, 0.47990640803167306, 1.0, 0.09984452286269897], [0.11188766703422799, -0.7172529382519395, -0.3521024461597403, 0.04797381132680503, 0.09984452286269897, 1.0]]
